# Abalone age predictor using tf.layers

This tutorial covers how to create your own training script using the building
blocks provided in `tf.layers`, which will predict the ages of
[abalones](https://en.wikipedia.org/wiki/Abalone) based on their physical
measurements. You'll learn how to do the following:

*   Instantiate an `sagemaker.Estimator`
*   Construct a custom model function
*   Configure a neural network using `tf.feature_column` and `tf.layers`
*   Choose an appropriate loss function from `tf.losses`
*   Define a training op for your model
*   Generate and return predictions

## An Abalone Age Predictor

It's possible to estimate the age of an
[abalone](https://en.wikipedia.org/wiki/Abalone) (sea snail) by the number of
rings on its shell. However, because this task requires cutting, staining, and
viewing the shell under a microscope, it's desirable to find other measurements
that can predict age.

The [Abalone Data Set](https://archive.ics.uci.edu/ml/datasets/Abalone) contains
the following
[feature data](https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.names)
for abalone:

| Feature        | Description                                               |
| -------------- | --------------------------------------------------------- |
| Length         | Length of abalone (in longest direction; in mm)           |
| Diameter       | Diameter of abalone (measurement perpendicular to length; in mm)|
| Height         | Height of abalone (with its meat inside shell; in mm)     |
| Whole Weight   | Weight of entire abalone (in grams)                       |
| Shucked Weight | Weight of abalone meat only (in grams)                    |
| Viscera Weight | Gut weight of abalone (in grams), after bleeding          |
| Shell Weight   | Weight of dried abalone shell (in grams)                  |

The label to predict is number of rings, as a proxy for abalone age.

### Set up the environment

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

### Upload the data to a S3 bucket

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-abalone')

**sagemaker_session.upload_data** will upload the abalone dataset from your machine to a bucket named **sagemaker-{region}-{your aws account number}**, if you don't have this bucket yet, sagemaker_session will create it for you.

## Complete source code
Here is the full code for the network model:

In [ ]:
!cat 'abalone.py'

# Submitting script for training

In [ ]:
from sagemaker.tensorflow import TensorFlow

abalone_estimator = TensorFlow(entry_point='abalone.py',
                               role=role,
                               training_steps= 100,                                  
                               evaluation_steps= 100,
                               hyperparameters={'learning_rate': 0.001},
                               train_instance_count=1,
                               train_instance_type='ml.c4.xlarge')

abalone_estimator.fit(inputs)

`estimator.fit` will deploy a script in a container for training and returs the SageMaker model name using the following arguments:

*   **`entry_point="abalone.py"`** The path to the script that will be deployed to the container.
*   **`training_steps=100`** The number of training steps of the training job.
*   **`evaluation_steps=100`** The number of evaluation steps of the training job.
*   **`role`**. AWS role that gives your account access to SageMaker training and hosting
*   **`hyperparameters={'learning_rate' : 0.001}`**. Training hyperparameters. 

Running the code block above will do the following actions:
* deploy your script in a container with tensorflow installed
* copy the data from the bucket to the container
* instantiate the tf.estimator
* train the estimator with 100 training steps
* save the estimator model

# Submiting a trained model for hosting


In [ ]:
abalone_predictor = abalone_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

`abalone_estimator.deploy` deploys the trained model in a container ready for production.

# Invoking the endpoint

In [ ]:
import tensorflow as tf
import numpy as np

prediction_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=os.path.join('data/abalone_predict.csv'),
    target_dtype=np.int,
    features_dtype=np.float32
)

data = prediction_set.data[0]
tensor_proto = tf.make_tensor_proto(values=np.asarray(data), shape=[1, len(data)], dtype=tf.float32)

In [ ]:
abalone_predictor.predict(tensor_proto)

# Deleting the endpoint

In [ ]:
sagemaker.Session().delete_endpoint(abalone_predictor.endpoint)